In [ ]:
import pandas as pd
df_reduced = pd.read_csv("autoencoder_dataset.csv")
df_reduced.head()

,encoded_1,encoded_2,encoded_3,encoded_4,encoded_5,encoded_6,encoded_7,encoded_8,encoded_9,encoded_10,...,encoded_92,encoded_93,encoded_94,encoded_95,encoded_96,encoded_97,encoded_98,encoded_99,encoded_100,Label
0,0.00000,0.00000,0.0000,0.0000,0.00000,0.0,0.000000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.0000,549.19870,0.0000,1678.4856,0.0,929.9107,0.0,cancerous
1,0.00000,0.00000,0.0000,1315.5032,0.00000,0.0,0.000000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.0000,705.28380,1265.4187,0.0000,0.0,0.0000,0.0,cancerous
2,0.00000,69.50759,1502.5161,0.0000,0.00000,0.0,0.000000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.0000,0.00000,0.0000,0.0000,0.0,0.0000,0.0,cancerous
3,185.81334,0.00000,0.0000,0.0000,256.93192,0.0,48.212173,460.39550,146.50910,0.00000,...,152.976260,0.00000,115.7502,89.84800,0.0000,0.0000,0.0,0.0000,0.0,cancerous
4,0.00000,700.31600,0.0000,0.0000,0.00000,0.0,0.000000,99.23599,202.07448,148.24594,...,59.200123,262.94797,0.0000,259.64282,0.0000,0.0000,0.0,0.0000,0.0,cancerous


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso

# Check the original shape of the reduced dataset
print("Original shape of the reduced dataset:", df_reduced.shape)

# Separate features and target
X = df_reduced.drop(columns=['Label'])
y = df_reduced['Label']

# Encode the target variable 'y' (Label Encoding)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 1: Filter Method - Mutual Information
print("Step 1: Applying Filter Method (Mutual Information)")
from sklearn.feature_selection import mutual_info_classif
mutual_info = mutual_info_classif(X, y_encoded)
mi_threshold = 0.01  # Set a threshold for feature relevance
selected_features_filter = X.columns[mutual_info > mi_threshold]
X_filtered = X[selected_features_filter]
print(f"Number of features after filter method: {X_filtered.shape[1]}")

# Step 2: Wrapper Method - RFE with Random Forest
print("Step 2: Applying Wrapper Method (RFE with Random Forest)")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filtered)  # Scale features for RFE
rf_model = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rf_model, n_features_to_select=50, step=10)
rfe.fit(X_scaled, y_encoded)
selected_features_rfe = selected_features_filter[rfe.support_]
X_rfe = X_filtered[selected_features_rfe]
print(f"Number of features after wrapper method: {X_rfe.shape[1]}")

# Step 3: Embedded Method - Lasso (L1 Regularization)
print("Step 3: Applying Embedded Method (LASSO)")
lasso = Lasso(alpha=0.01, random_state=42)
lasso.fit(X_rfe, y_encoded)  # Use encoded target variable
lasso_selected = np.where(lasso.coef_ != 0)[0]  # Features with non-zero coefficients
final_selected_features = X_rfe.columns[lasso_selected]
X_final = X_rfe[final_selected_features]
print(f"Number of features after embedded method: {X_final.shape[1]}")

# Combine the final selected features with the target variable 'Label'
final_df = pd.concat([X_final, y], axis=1)  # Ensure the target variable is kept

# Show the final reduced dataset as CSV output
print("Final reduced dataset with selected features and target variable:")
#print(final_df.to_csv(index=False))  # Display the output as CSV
print(final_df)


Original shape of the reduced dataset: (21, 101)
Step 1: Applying Filter Method (Mutual Information)
Number of features after filter method: 42
Step 2: Applying Wrapper Method (RFE with Random Forest)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_rfe.py:300: UserWarning: Found n_features_to_select=50 > n_features=42. There will be no feature selection and all features will be kept.
  warnings.warn(


Number of features after wrapper method: 42
Step 3: Applying Embedded Method (LASSO)
Number of features after embedded method: 33
Final reduced dataset with selected features and target variable:
     encoded_1   encoded_4   encoded_8  encoded_11  encoded_15  encoded_16  \
0      0.00000     0.00000    0.000000    0.000000     0.00000    0.000000   
1      0.00000  1315.50320    0.000000    0.000000     0.00000    0.000000   
2      0.00000     0.00000    0.000000    0.000000     0.00000    0.000000   
3    185.81334     0.00000  460.395500    0.000000     0.00000  416.146330   
4      0.00000     0.00000   99.235990    0.000000     0.00000    0.000000   
5   1262.60010     0.00000    0.000000    0.000000     0.00000    0.000000   
6      0.00000     0.00000    0.000000  319.558750   302.24435    0.000000   
7      0.00000     0.00000    0.000000    0.000000  2132.31670    0.000000   
8      0.00000     0.00000    0.000000    0.000000     0.00000    0.000000   
9    194.35430   718.659

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.772e-03, tolerance: 4.952e-04
  model = cd_fast.enet_coordinate_descent(


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso

# Check the original shape of the reduced dataset
print("Original shape of the reduced dataset:", df_reduced.shape)

# Separate features and target
X = df_reduced.drop(columns=['Label'])
y = df_reduced['Label']

# Encode the target variable 'y' (Label Encoding)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 1: Filter Method - Mutual Information
print("Step 1: Applying Filter Method (Mutual Information)")
from sklearn.feature_selection import mutual_info_classif
mutual_info = mutual_info_classif(X, y_encoded)
mi_threshold = 0.01  # Set a threshold for feature relevance
selected_features_filter = X.columns[mutual_info > mi_threshold]
X_filtered = X[selected_features_filter]
print(f"Number of features after filter method: {X_filtered.shape[1]}")

# Step 2: Wrapper Method - RFE with Random Forest
print("Step 2: Applying Wrapper Method (RFE with Random Forest)")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filtered)  # Scale features for RFE
rf_model = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rf_model, n_features_to_select=50, step=10)
rfe.fit(X_scaled, y_encoded)
selected_features_rfe = selected_features_filter[rfe.support_]
X_rfe = X_filtered[selected_features_rfe]
print(f"Number of features after wrapper method: {X_rfe.shape[1]}")

# Step 3: Embedded Method - Lasso (L1 Regularization)
print("Step 3: Applying Embedded Method (LASSO)")
lasso = Lasso(alpha=0.01, random_state=42)
lasso.fit(X_rfe, y_encoded)  # Use encoded target variable
lasso_selected = np.where(lasso.coef_ != 0)[0]  # Features with non-zero coefficients
final_selected_features = X_rfe.columns[lasso_selected]
X_final = X_rfe[final_selected_features]
print(f"Number of features after embedded method: {X_final.shape[1]}")

# Combine the final selected features with the target variable 'Label'
final_df = pd.concat([X_final, y], axis=1)  # Ensure the target variable is kept

# Save the final DataFrame to a CSV file
output_filename = 'selected_features_dataset.csv'
final_df.to_csv(output_filename, index=False)
print(f"\nDataset has been saved as '{output_filename}'")

# Display the first few rows of the saved dataset
print("\nFirst few rows of the saved dataset:")
print(final_df.head())

# Display dataset information
print("\nDataset information:")
print(f"Number of rows: {final_df.shape[0]}")
print(f"Number of columns: {final_df.shape[1]}")
print(f"Columns: {', '.join(final_df.columns)}")

Original shape of the reduced dataset: (21, 101)
Step 1: Applying Filter Method (Mutual Information)
Number of features after filter method: 55
Step 2: Applying Wrapper Method (RFE with Random Forest)
Number of features after wrapper method: 50
Step 3: Applying Embedded Method (LASSO)
Number of features after embedded method: 37

Dataset has been saved as 'selected_features_dataset.csv'

First few rows of the saved dataset:
   encoded_1  encoded_4  encoded_8  encoded_11  encoded_13  encoded_17  \
0    0.00000     0.0000    0.00000         0.0      0.0000     0.00000   
1    0.00000  1315.5032    0.00000         0.0      0.0000     0.00000   
2    0.00000     0.0000    0.00000         0.0      0.0000     0.00000   
3  185.81334     0.0000  460.39550         0.0      0.0000   313.12375   
4    0.00000     0.0000   99.23599         0.0    534.2414     0.00000   

   encoded_18  encoded_21  encoded_22  encoded_23  ...  encoded_69  \
0     0.00000      0.0000    0.000000     0.00000  ...   

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.489e-03, tolerance: 4.952e-04
  model = cd_fast.enet_coordinate_descent(


In [ ]:
print(final_df)

     encoded_1   encoded_4   encoded_8  encoded_11  encoded_13  encoded_17  \
0      0.00000     0.00000    0.000000    0.000000     0.00000     0.00000   
1      0.00000  1315.50320    0.000000    0.000000     0.00000     0.00000   
2      0.00000     0.00000    0.000000    0.000000     0.00000     0.00000   
3    185.81334     0.00000  460.395500    0.000000     0.00000   313.12375   
4      0.00000     0.00000   99.235990    0.000000   534.24140     0.00000   
5   1262.60010     0.00000    0.000000    0.000000     0.00000     0.00000   
6      0.00000     0.00000    0.000000  319.558750   492.86234   271.91568   
7      0.00000     0.00000    0.000000    0.000000     0.00000  1004.79240   
8      0.00000     0.00000    0.000000    0.000000     0.00000     0.00000   
9    194.35430   718.65936    0.000000  200.597100   407.91858   114.94674   
10   906.38710     0.00000    0.000000    0.000000     0.00000     0.00000   
11     0.00000     0.00000    0.000000    0.000000     0.00000  